# Import Dependencies and CSV Files

In [1]:
import pandas as pd
import numpy as np
import re
import time

from pathlib import Path
from collections import Counter

In [2]:
# File Paths
movies = "Starting_Data/movies.csv"

# Read CSV Files
movie_df = pd.read_csv(movies, low_memory=False)

In [3]:
movie_df.head(20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# Split Movie Title and Year, Create Separate Columns

In [4]:
# pattern = '\((\d{4})\)'
# years = movie_df["title"].str.extract(pattern, expand=False)

pattern_2 = '(.*\w*)(?:\s\((\d{4})\))'
title = movie_df["title"].str.extract(pattern_2, expand=False)
title["Title"] = title[0]
title["Year"] = title[1]
movie_df["Title"] =  title["Title"]
movie_df["Year"] = title["Year"]
movie_df.head(20)

,movieId,title,genres,Title,Year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995
5,6,Heat (1995),Action|Crime|Thriller,Heat,1995
6,7,Sabrina (1995),Comedy|Romance,Sabrina,1995
7,8,Tom and Huck (1995),Adventure|Children,Tom and Huck,1995
8,9,Sudden Death (1995),Action,Sudden Death,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,GoldenEye,1995


# Separate Genres Into Lists

In [5]:
movie_df["genres"] = movie_df["genres"].str.split('|', expand=False)
movie_df = movie_df.drop(columns="title")
Genres = movie_df["genres"].to_list()

# Constructing Function To Split Genre List Into Separate Columns With Boolean Entries

In [6]:
Genres_df = pd.DataFrame(columns=['mg_id', 'genre_id', "movie_id"])
Genres_df["movieId"] = movie_df["movieId"]
Genres_df.head()

,mg_id,genre_id,movie_id,movieId
0,NaN,NaN,NaN,1
1,NaN,NaN,NaN,2
2,NaN,NaN,NaN,3
3,NaN,NaN,NaN,4
4,NaN,NaN,NaN,5


# Function

In [7]:
def genre_split(df_1, df_2):
    i = -1
    x = 0
    for row in df_1["genres"]:
        x += 1
        if x in df_1["movieId"]:
            for line in row:
                i += 1
                if line != "(no genres listed)":
                    if line == "Adventure":
                        df_2.at[i, "genre_id"] = 1
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Action":
                        df_2.at[i, "genre_id"] = 2
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Animation":
                        df_2.at[i, "genre_id"] = 3
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Children":
                        df_2.at[i, "genre_id"] = 4
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Comedy":
                        df_2.at[i, "genre_id"] = 5
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Crime":
                        df_2.at[i, "genre_id"] = 6
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Documentary":
                        df_2.at[i, "genre_id"] = 7
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Drama":
                        df_2.at[i, "genre_id"] = 8
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Fantasy":
                        df_2.at[i, "genre_id"] = 9
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Film-Noir":
                        df_2.at[i, "genre_id"] = 10
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Horror":
                        df_2.at[i, "genre_id"] = 11
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "IMAX":
                        df_2.at[i, "genre_id"] = 12
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Musical":
                        df_2.at[i, "genre_id"] = 13
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Mystery":
                        df_2.at[i, "genre_id"] = 14
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Romance":
                        df_2.at[i, "genre_id"] = 15
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Sci-Fi":
                        df_2.at[i, "genre_id"] = 16
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Thriller":
                        df_2.at[i, "genre_id"] = 17
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "War":
                        df_2.at[i, "genre_id"] = 18
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    elif line == "Western":
                        df_2.at[i, "genre_id"] = 19
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                    else:
                        df_2.at[i, "genre_id"] = 20      
                        df_2.at[i, "movie_id"] = x
                        df_2.at[i, "mg_id"] = i
                else:
                    df_2.at[i, "genre_id"] = 20
                    df_2.at[i, "movie_id"] = x
                    df_2.at[i, "mg_id"] = i
        else:
            continue   
    return df_2.head()

In [8]:
genre_split(movie_df, Genres_df)

,mg_id,genre_id,movie_id,movieId
0,0,1,1,1.0
1,1,3,1,2.0
2,2,4,1,3.0
3,3,5,1,4.0
4,4,9,1,5.0


In [9]:
Genres_df = Genres_df.drop(columns=["movieId"])
Genres_df.head(20)

,mg_id,genre_id,movie_id
0,0,1,1
1,1,3,1
2,2,4,1
3,3,5,1
4,4,9,1
5,5,1,2
6,6,4,2
7,7,9,2
8,8,5,3
9,9,15,3


# Clean Up Movie Dataframe

In [10]:
movie_df["movie_id"] = movie_df["movieId"]
movie_df["name"] = movie_df["Title"]
movie_df["year"] = movie_df["Year"]
movie_df = movie_df.drop(columns=["Title", "Year", "movieId", "genres"])
movie_df = movie_df.loc[:, ["movie_id", "name", "year"]]

In [11]:
movie_df.head()

,movie_id,name,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


# Write DataFrame To CSV File

In [13]:
Genres_df.to_csv("Active_Data/movie_genres.csv", index=False)
movie_df.to_csv("Active_Data/movies.csv", index=False)